#Data Acquisition


In [2]:
!pip install scikit-surprise # Install the scikit-surprise package. It is named surprise, but you must pip install scikit-surprise
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

# Split
from sklearn.model_selection import train_test_split

from surprise import Reader, Dataset, SVD # Now you can import from surprise
from surprise.model_selection import cross_validate

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Kareem\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Kareem\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Karee

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

# Load Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import io

# Load the dataset
df = pd.read_csv(io.BytesIO(uploaded['ratings_Electronics (1).csv']))

In [ ]:
df = pd.read_csv('ratings_Electronics (1).csv', names=['userId', 'productId','rating','timestamp']) # remove the file path as the file is assumed to be in the current working directory.

# Data Cleaning & EDA

In [ ]:
df.head()

,userId,productId,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [ ]:
print("Total Reviews:",df.shape[0])
print("Total Columns:",df.shape[1])

Total Reviews: 7824482
Total Columns: 4


In [ ]:
# Taking subset of the dataset
df = df.iloc[:5000,0:]

In [ ]:
print("Total Reviews:",df.shape[0])
print("Total Columns:",df.shape[1])

Total Reviews: 5000
Total Columns: 4


In [ ]:
print("Total number of ratings :",df.rating.nunique())
print("Total number of users   :", df.userId.nunique())
print("Total number of products  :", df.productId.nunique())

Total number of ratings : 5
Total number of users   : 4929
Total number of products  : 299


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     5000 non-null   object 
 1   productId  5000 non-null   object 
 2   rating     5000 non-null   float64
 3   timestamp  5000 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 156.4+ KB


In [ ]:
# Check missing value
df.isnull().sum()

,0
userId,0
productId,0
rating,0
timestamp,0


In [ ]:
# Check Duplicate data
df[df.duplicated()].any()

,0
userId,False
productId,False
rating,False
timestamp,False


In [ ]:
# rating describe summary
df.describe()['rating']

,rating
count,5000.000000
mean,3.986600
std,1.411814
min,1.000000
25%,3.000000
50%,5.000000
75%,5.000000
max,5.000000


In [ ]:
print("Unique value of Rating:",df.rating.unique())

Unique value of Rating: [5. 1. 3. 2. 4.]


In [ ]:
# Find the minimum and maximum ratings
print('Minimum rating is: %d' %(df.rating.min()))
print('Maximum rating is: %d' %(df.rating.max()))

Minimum rating is: 1
Maximum rating is: 5


In [ ]:
# Convert userId and productId to string
df['userId'] = df['userId'].astype(str)
df['productId'] = df['productId'].astype(str)

# Convert rating to float
df['rating'] = df['rating'].astype(float)

# Convert timestamp to datetime format
import pandas as pd
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')  # Assuming timestamp is in seconds

In [ ]:
# Filter out ratings outside 1-5 range
df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]

In [ ]:
df = df.drop_duplicates()

# Feature Engineering

In [ ]:
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

In [ ]:
# Number of ratings per user
user_ratings_count = df.groupby('userId').size()
df['user_ratings_count'] = df['userId'].map(user_ratings_count)

# Number of ratings per product
product_ratings_count = df.groupby('productId').size()
df['product_ratings_count'] = df['productId'].map(product_ratings_count)

In [ ]:
user_mean_rating = df.groupby('userId')['rating'].transform('mean')
df['rating_normalized'] = df['rating'] - user_mean_rating

In [ ]:
df.to_csv('cleaned_amazon_reviews_with_features.csv', index=False)

# Recommender System type

Setup

In [ ]:
!pip install scikit-surprise
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357279 sha256=791eefc8423362f4ea8bf1176945137b17132d0fc6574795603995d1f2a8013a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


Load Data

In [ ]:
# Load your cleaned dataset
file_path = '/content/cleaned_amazon_reviews_with_features.csv'
df = pd.read_csv(file_path)

# Preview the data
print(df.head())

           userId   productId  rating   timestamp  year  month  day  \
0   AKM1MP6P0OYPR  0132793040     5.0  2013-04-13  2013      4   13   
1  A2CX7LUOHB2NDG  0321732944     5.0  2012-07-01  2012      7    1   
2  A2NWSAGRHCP8N5  0439886341     1.0  2013-04-29  2013      4   29   
3  A2WNBOD3WNDNKT  0439886341     3.0  2013-07-22  2013      7   22   
4  A1GI0U4ZRJA8WN  0439886341     1.0  2012-04-18  2012      4   18   

   user_ratings_count  product_ratings_count  rating_normalized  
0                   1                      1                0.0  
1                   1                      1                0.0  
2                   1                      3                0.0  
3                   1                      3                0.0  
4                   1                      3                0.0  


# Collaborative Filtering

In [ ]:
# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'productId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
# Train the SVD model
svd = SVD()
svd.fit(trainset)

# Make predictions on the test set
predictions = svd.test(testset)

# Evaluate the model using RMSE
accuracy.rmse(predictions)

RMSE: 1.3553


1.3552824882835581

In [ ]:
# Train the k-NN model
from surprise import KNNBasic # import the KNNBasic class
knn = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
knn.fit(trainset)

# Make predictions on the test set
predictions_knn = knn.test(testset)

# Evaluate the k-NN model
accuracy.rmse(predictions_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3880


1.3880459331855302

# Content-Based Filtering

In [ ]:
user_product_matrix = df.pivot(index='userId', columns='productId', values='rating').fillna(0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity # import the cosine_similarity function

user_product_matrix = df.pivot(index='userId', columns='productId', values='rating').fillna(0)

product_similarity = cosine_similarity(user_product_matrix.T)
product_similarity_df = pd.DataFrame(product_similarity, index=user_product_matrix.columns, columns=user_product_matrix.columns)

In [ ]:
def recommend_products(product_id, num_recommendations=5):
    # Check if the product_id exists in the DataFrame
    if product_id in product_similarity_df.columns:
        similar_products = product_similarity_df[product_id].sort_values(ascending=False).iloc[1:num_recommendations+1]
        return similar_products
    else:
        return f"Product ID '{product_id}' not found in the dataset."

# Example recommendation for a specific product
recommend_products('0321732944')

,0321732944
productId,
7214047977,0.0
7200599557,0.0
7108037661,0.0
7040034018,0.0
6911584914,0.0


In [ ]:
# Calculate RMSE for content-based filtering by predicting based on similarity
from sklearn.metrics import mean_squared_error # import the mean_squared_error function

def predict_content_based(user_id, product_id):
    similar_products = product_similarity_df[product_id].sort_values(ascending=False)
    similar_products_ratings = user_product_matrix.loc[user_id, similar_products.index]
    similar_products_ratings = similar_products_ratings[similar_products_ratings > 0]
    if not similar_products_ratings.empty:
        return similar_products_ratings.mean()
    else:
        return user_product_matrix.loc[user_id].mean()

# Example: calculate RMSE for content-based filtering
y_true = []
y_pred = []

for user_id, product_id, rating in df[['userId', 'productId', 'rating']].values:
    if product_id in product_similarity_df.columns:
        y_true.append(rating)
        y_pred.append(predict_content_based(user_id, product_id))

print("Content-based RMSE:", mean_squared_error(y_true, y_pred, squared=False))

Content-based RMSE: 0.09629814813034222


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
